In [280]:
import json
import collections
from urllib.request import urlopen
import re
import romkan
import pickle
import os

In [281]:
def init():
    global articles, word_list, list_of_articles, list_of_new_articles
    articles = collections.namedtuple('articles', 'title, date, url, text')
    word_list = list()
    list_of_articles = list()
    list_of_new_articles = list()

def update():
    global articles, word_list, list_of_articles, list_of_new_articles
    request = urlopen("http://www3.nhk.or.jp/news/easy/news-list.json")
    data = json.loads(request.read().decode('utf-8-sig'))

    if os.path.isfile('word_list.p'):
        word_list = pickle.load(open('word_list.p', 'rb'))
        list_of_articles = pickle.load(open('list_of_articles.p', 'rb'))

    for date in data[0].keys():
        article = articles(data[0][date][0]['title_with_ruby'], date, 'http://www3.nhk.or.jp/news/easy/'+ data[0][date][0]['news_id'] +'/'+ data[0][date][0]['news_id'] +'.out.json')
        if article not in list_of_articles:
            list_of_articles.append(article)
            list_of_new_articles.append(article)

    phrase = ''

    for article in list_of_articles:
        article_json = json.loads(urlopen(article[2]).read().decode('utf-8-sig')) #loads the json for the article
        for pronunciation in article_json['morph']:
            furi_list = pronunciation.get('ruby', [dict()])
            for furi_dict in furi_list:
                furigana = furi_dict.get('r', furi_dict.get('s', ''))
                if re.match(u'[\u3040-\u309F]+', furigana) != None:
                    hiragana = re.match(u'[\u3040-\u309F]+', furigana) #hiragana
                    if hiragana != None: #check for hiragana
                        phrase += hiragana.string
                elif phrase != '':
                    word_list.append(phrase)
                    phrase = ''

    pickle.dump(word_list, open("word_list.p", "wb"))
    pickle.dump(list_of_articles, open("list_of_articles.p", "wb"))

In [282]:
init()

In [283]:
update()